<a href="https://colab.research.google.com/github/Agustin-Galarza/tp_nlp/blob/main/tp_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>